<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install morfeusz2

     |████████████████████████████████| 8.5 MB 7.9 MB/s 


In [ ]:
# instalacja morfeusza
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz' pl_spacy_model_morfeusz-0.1.3.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.utils import simple_preprocess
from collections import Counter
import gc
import morfeusz2
import spacy
from spacy.lang.pl.stop_words import STOP_WORDS

In [47]:
# Wyświetlanie szerszych kolumn, aby nie skracać wyświetlanych nazw tematów
pd.set_option('max_colwidth', 400)

In [48]:
morf = morfeusz2.Morfeusz()

In [49]:
df = pd.read_excel('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/budzet_obywatelski.xlsx', sheet_name='Arkusz1')
df.rename(columns={'rok ': 'rok', 'miasto ':'miasto'}, inplace=True)
df = df[~df['nazwa'].isna()]
df['miasto'] = df['miasto'].map(lambda x: x.upper())
# Usunięcie pustych wpisów

# Indeksy do usunięcia
rem_index = df[df['nazwa'].isnull()].index.tolist()
rem_index += df[df['nazwa']=='bd'].index.tolist()

df.drop(df.index[rem_index], inplace=True)
# Usunięcie duplikatów
df = df.drop_duplicates()
df['nazwa'] = df['nazwa'].apply(str)
#usuniecie kolumny wszystko
if 'WSZYSTKO' in df: del df['WSZYSTKO']
df.sample(10)

,rok,miasto,nazwa
11407,2019,LUBLIN,"Rewitalizacja terenu SP16 - Z klasą i z rodziną czas płynie zdrowo na placu zabaw pod chmurką, zbudowanym odlotowo! Budowa placu zabaw."
17230,2019,PŁOCK,Centrum Doświadczania Świata Kraina Zmysłów
16230,2019,KRAKÓW,Dęby zamiast pomników
8955,2018,WROCŁAW,Kuźnicki Psi Park Zabaw
20588,2019,WARSZAWA,Nowe przejście dla pieszych na Odolanach
1347,2020,GDAŃSK,Góra z górą ...
1405,2020,GDAŃSK,Nartorolki dla Gdańska. Alternatywa dla narciarstwa biegowego. Gdańsk jako stolica nartorolek w Polsce.
6749,2018,KALISZ,Koncertowa majówka na Rajskiej Polanie przy ulicy Wał Matejki.
15891,2018,KRAKÓW,Tradycja i nowocześność‐ stroje Bronowickie dla SP 50.
12227,2018/19,ŁÓDŹ,Wypoczynek na Żubardziu - relaks i podwórkowe zabawy sportowe.


In [50]:
df.loc[(df['miasto'] == 'WARSZAWA'), "czy_WAW"] = 1
df.loc[df["czy_WAW"].isnull(),  "czy_WAW"] = 0
df["czy_WAW"].value_counts()

0.0    18431
1.0     4599
Name: czy_WAW, dtype: int64

# WARSZAWA

In [51]:
df_waw = df[df['miasto'] == 'WARSZAWA']
df_waw.head()

,rok,miasto,nazwa,czy_WAW
18418,2018,WARSZAWA,Domki dla jeży,1.0
18419,2018,WARSZAWA,Wolskie lato filmowe,1.0
18420,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,1.0
18421,2018,WARSZAWA,Czysta Wisła na Pradze Południe,1.0
18422,2018,WARSZAWA,Wola na 2 koła! Kontraruch rowerowy w całej dzielnicy,1.0


In [52]:
data_waw =[]

In [53]:
for rows in range(1,df_waw.shape[0]):
  text = df_waw.iloc[rows, 2]
  miasto = df_waw.iloc[rows, 1]
  rok = df_waw.iloc[rows, 0]
  analysis = morf.analyse(text)
  for interpretation in analysis:
      start_ind, end_ind, interp = interpretation
      form, lemma, tag, qualifiers, _ = interpretation[2]
      data_waw.append((rok, miasto, text, start_ind, end_ind, form,lemma,tag,qualifiers,_))

In [57]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
df_waw_morf4 = pd.DataFrame(data_waw, columns=cols)
df_waw_morf4.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:acc:m2.m3.f.n:pos,[],[]
1,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
2,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:acc:n:pos,[],[]
3,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:nom.voc:n:pos,[],[]
4,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolskie,subst:pl:nom.acc.voc:n:pt,[nazwa_geograficzna],[]
5,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolska,subst:pl:nom.acc.voc:f,[nazwisko],[]
6,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolski,depr:pl:nom.acc.voc:m2,[nazwisko],[]
7,2018,WARSZAWA,Wolskie lato filmowe,1,2,lato,lato,subst:sg:nom.acc.voc:n:ncol,[nazwa_pospolita],[]
8,2018,WARSZAWA,Wolskie lato filmowe,2,3,filmowe,filmowy,adj:pl:acc:m2.m3.f.n:pos,[],[]
9,2018,WARSZAWA,Wolskie lato filmowe,2,3,filmowe,filmowy,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]


In [58]:
df_waw_morf4.shape

(89425, 10)

In [59]:
df_waw_morf4['qualifiers'].map(lambda x: str(x)).value_counts()

[]                                                             47582
['nazwa_pospolita']                                            30221
['nazwisko']                                                    5890
['nazwa_geograficzna']                                          3405
['nazwa_geograficzna', 'nazwisko']                              1174
['imię']                                                         437
['człon_nazwy_geograficznej']                                    224
['nazwa_organizacji']                                            166
['imię', 'nazwisko']                                             137
['nazwa_firmy']                                                   35
['imię', 'nazwa_geograficzna']                                    28
['nazwa pospolita']                                               19
['człon_nazwy_geograficznej', 'nazwisko']                         19
['nazwa_instytucji']                                              18
['człon_nazwy_organizacji']       

In [60]:
df_waw_morf4['_'].map(lambda x: str(x)).value_counts().head(60)

[]                          85243
['daw.']                      842
['pot.']                      638
['przest.']                   519
['hom.']                      408
['muz.']                      382
['zool.']                     144
['książk.']                   142
['gwar.']                     108
['jęz.']                       97
['rzad.']                      88
['sport.']                     80
['bot.']                       73
['pogard.']                    37
['środ.']                      36
['z_d.']                       32
['arch.,char.']                32
['anat.']                      30
['karc.']                      26
['wojsk.']                     22
['daw.,praw.']                 21
['szkol.']                     19
['wulg.']                      18
['przest.,techn.']             18
['hist.']                      17
['pot.,środ.']                 16
['komp.']                      15
['przen.']                     15
['geogr.']                     14
['pot.,bot.'] 

In [61]:
czasownik = ['fin', 'bedzie', 'aglt', 'praet', 'impt', 'imps', 'inf', 'pcon', 'pant', 'ger', 'pact', 'ppas']
df_waw_morf4_verb = df_waw_morf4[df_waw_morf4['tag'].str.contains('|'.join(czasownik))]

In [62]:
df_waw_morf4_verb.lemma.value_counts().head(40)

zająć           365
zabawić         216
spotkać         185
przejść         148
oświetlić       127
zielenić        111
wzdłużyć        102
zakupić          99
nasadzić         93
być              61
skrzyżować       46
doświetlić       46
żyć              46
chronić          41
urządzić         35
otworzyć         34
programować      34
pić              34
szkolić          32
ratować          27
dojść            27
śmiecić          26
ćwiczyć          26
doposażyć        26
woleć            26
rozwijać         25
przystać:Vp      24
przestrzenić     23
mieć             22
parkować         21
grać             21
czytać           20
wesprzeć         19
brakować         19
zwalniać         19
zjednoczyć       18
wyposażyć        18
posadzić         18
ogrodzić         16
pływać           16
Name: lemma, dtype: int64

In [63]:
df_waw_morf4[df_waw_morf4['_'].map(set(['zool.']).issubset)]

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
13,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
17,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,3,4,wróble,wróbel,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
104,2018,WARSZAWA,Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
108,2018,WARSZAWA,Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe,3,4,wróble,wróbel,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
215,2018,WARSZAWA,Chronimy jerzyki i wróble na Bielanach - skrzynki lęgowe,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
...,...,...,...,...,...,...,...,...,...,...
86153,2020,WARSZAWA,Muzyka nad Stawami Cietrzewia,3,4,Cietrzewia,cietrzew,subst:sg:gen.acc:m2,[nazwa_pospolita],[zool.]
88095,2020,WARSZAWA,"„Chronimy ptaki, nietoperze i wiewiórki - zawieszenie skrzynek lęgowych oraz wykonanie tratw dla ptactwa wodnego”",4,5,nietoperze,nietoperz,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
88360,2020,WARSZAWA,Wieże lęgowe dla jerzyków,3,4,jerzyków,jerzyk,subst:pl:gen:m2,[nazwa_pospolita],[zool.]
88420,2020,WARSZAWA,Chronimy jerzyki i wróble na Żoliborzu - montaż skrzynek lęgowych,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]


In [64]:
df_waw_morf4.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_WAW.csv', index=False)

## Reszta miast

In [65]:
df_roc = df[df['miasto'] != "WARSZAWA"]
df_roc.head(10)

,rok,miasto,nazwa,czy_WAW
0,2020,GDYNIA,Projektistatus,0.0
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0.0
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację,0.0
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",0.0
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną,0.0
5,2020,GDYNIA,Cotygodniowe weekendowe lekcje tańca towarzyskiego dla dorosłych,0.0
6,2020,GDYNIA,Meble miejskie zaprojektowane przez dzieci. Podziemia Hal Targowych w Gdyni.,0.0
7,2020,GDYNIA,"Odnowienie podwórek w obrębie ulic Płk. Dąbka, Kwiatkowskiego, Sucharskiego, Ledóchowskiego i Maciejewicza",0.0
8,2020,GDYNIA,Poprawa atrakcyjności skweru przy skrzyżowaniu Wielkopolska-Łowicka-Płocka przez nasadzenie roślin ozdobnych,0.0
9,2020,GDYNIA,Radarowe wyświetlacze prędkości na ul. Morskiej,0.0


In [66]:
data_roc2 =[]

In [67]:
for rows in range(1,df_roc.shape[0]):
  text = df_roc.iloc[rows, 2]
  miasto = df_roc.iloc[rows, 1]
  rok = df_roc.iloc[rows, 0]
  analysis = morf.analyse(text)
  for interpretation in analysis:
      start_ind, end_ind, interp = interpretation
      form, lemma, tag, qualifiers, _ = interpretation[2]
      data_roc2.append((rok, miasto, text, start_ind, end_ind, form,lemma,tag,qualifiers,_))

In [69]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
df_roc_morf5 = pd.DataFrame(data_roc2, columns=cols)
df_roc_morf5.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,[nazwa_pospolita],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,[nazwa_pospolita],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,[nazwa_pospolita],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]
5,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:inst,[],[]
6,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",3,4,Estakadą,estakada,subst:sg:inst:f,[nazwa_pospolita],[]
7,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",4,5,Kwiatkowskiego,Kwiatkowski:Sm1,subst:sg:gen.acc:m1,[nazwisko],[]
8,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:C,conj,[],[]
9,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:I,interj,[],[]


In [70]:
df_roc_morf5.shape

(419564, 10)

In [72]:
df_roc_morf5[df_roc_morf5['_'].map(set(['zool.']).issubset)].sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
97445,2020,KATOWICE,"Zielony przystanek jako element walki ze smogiem? Wyglada na to, ze tak. Coraz wiecej miast wprowadza takie rozwiazania - Biatystok, Kraków, Wroclaw, Siemiatycze. Oczywiscie rozwiazania sa rózne, wprowadza sie zazielenienia istniejacych wiat przystankowych, lub projektuje catkowicie nowe. Takie wiaty przystankowe pokazywane byty podczas XVII Samorzadowego Forum Kapitatu i Finansów W Katowicach...",122,123,jak,jak:S,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
384520,2020,NOWY SĄCZ,Słoneczna energia- panele fotowoltaiczne dla Szkoły Podstawowej nr 20 im. Stefana Kardynała Wyszyńskiego Z Oddziałami integracyjnymi W Nowym Sączu.,13,14,Kardynała,kardynał:Sm2,subst:sg:gen.acc:m2,[nazwa_pospolita],[zool.]
338160,2019,ŁÓDŹ,Ratujmy zabytkową studnię kręgową z żurawiem na ulicy Beskidzkiej.,5,6,żurawiem,żuraw:Sm2,subst:sg:inst:m2,[nazwa_pospolita],[zool.]
142081,2018,ŚWIĘTOCHŁOWICE,"Kompleksowa rozbudowa stref aktywnego wypoczynku i rekreacji poprzez wzbogacenie placu zabaw w rejonie ulic Chorzowskiej i Bieszczadzkiej, ze szczególnym uwzględnieniem urządzeń zabawowych dla dzieci niepełnosprawnych oraz budowa tzw. „siłowni pod chmurką” dla dorosłych przy boisku na ul. Ślęzan. Oczyszczenie oczek wodnych na Osiedlu Brzezina oraz tereny użytku ekologicznego tzw. „lasek chropa...",67,68,jak,jak:S,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
368178,2018,KRAKÓW,Do Wilgi ‐ promenada spacerowa,1,2,Wilgi,wilga,subst:pl:nom.acc.voc:f,[nazwa_pospolita],[zool.]
64669,2019,BYTOM,"#BO Rozbark - nowoczesne, bezpieczne, pelnowymiarowe, oswietlone boisko ze sztuczna nawierzchnia na stadionie GKS Rozbark przy ul. Chorzowskiej - Wykonanie pelnowymiarowego boiska, instalacjq energooszczednego oswietlenia pola gry, budowa ogrodzenia dookola boiska wraz Z wykonaniem pilko chwytów, wyznaczenie liniami boiska oraz dwóch boisk dla kategorii rozrywkowej Orlik, montaz dwóch bramek p...",66,67,Orlik,orlik:Sm2,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
138081,2019,SOSNOWIEC,Sosnowiec miasto przyjazne dla srodowiska. Potega miejskiej przyrody.,0,1,Sosnowiec,sosnowiec,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
371444,2019,KRAKÓW,TAK DLA WILGI,2,3,WILGI,wilga,subst:sg:gen:f,[nazwa_pospolita],[zool.]
291508,2018/19,ŁÓDŹ,"Ratujemy osiedlowe rzeźby-oczyszczenie i konserwacja rzeźby „Łabędzie"" na skwerze Kolbego przy ul. Tatrzańskiej.",9,10,Łabędzie,łabędź,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
186167,2019,KONIN,"Celem projektu jest budowa obiektu dajacego okazje do czynnego wypoczynku i promowanie zdrowego trybu zycia. Trening na swiezym powietrzu? Czemu nie! W swiecie petnym komputerów, braku aktywnosci fizycznej i ograniczonym kontakcie interpersonalnym budowa biezni rekreacyjnej jest szansa na aktywne spedzanie wolnego czasu Z jednoczesna korzyscia dla zdrowia. Éwiczenia na swiezym powietrzu przyno...",211,212,Orlik,orlik:Sm2,subst:sg:nom:m2,[nazwa_pospolita],[zool.]


In [73]:
df_roc_morf5.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_ROC.csv', index=False)

# 2018

In [74]:
df_2018 = df[df['rok'] == 2018]
df_2018.head(10)

,rok,miasto,nazwa,czy_WAW
654,2018,GDYNIA,Urzadzenie zieleni miejskiej miedzy ul. Dedala 6 a korona klifu,0.0
655,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0.0
656,2018,GDYNIA,Montaz fontann wody pitnej W ruchliwych punktach dzielnicy: przy placu zabaw oraz na poczatku sciezki rowerowej,0.0
657,2018,GDYNIA,"Rozbudowa placu zabaw dla dzieci mtodszych, starszych i mtodziezy przy ulicy Dedala 6-8",0.0
658,2018,GDYNIA,Montaz dwóch defibrylatorów - przy parafii przy ul. Rybaków oraz przy wspólnocie mieszkaniowej przy ul. Ikara 6,0.0
659,2018,GDYNIA,"Nasadzenia kwiatów i innych kompozycji roslinnych przy ulicach Dedala, Ikara, Zielonej i Rybaków",0.0
660,2018,GDYNIA,Oswietlenie boiska przy Szkole Podstawowej nr 28,0.0
661,2018,GDYNIA,Wykonanie ogólnodostepnego parkingu na terenie parafii Biskupa Józefa Sebastiana Pelczara,0.0
662,2018,GDYNIA,"Budowa rekreacyjnych torów do jazdy typu pumptrack wraz Z uzupelniajaca zielenia i miejscami do odpoczynku W poblizu przystanku autobusowego ""Gierdziejewskiego""",0.0
663,2018,GDYNIA,"Wymiana oswietlenia przy gtównych ulicach dzielnicy na oprawy LED oraz montaz solarnych tawek (ul. Okreina, Chwarznienska, Wiczlinska, Zaruskiego, Filipkowskiego i Staniszewskiego)",0.0


In [75]:
data_2018 =[]

In [76]:
def morfeusz_create_morf_table(df, data):
  for rows in range(1,df.shape[0]):
    text = df.iloc[rows, 2]
    miasto = df.iloc[rows, 1]
    rok = df.iloc[rows, 0]
    #print(text)
    analysis = morf.analyse(text)
    for interpretation in analysis:
        start_ind, end_ind, interp = interpretation
        form, lemma, tag, qualifiers, _ = interpretation[2]
        data.append((rok, miasto, text, start_ind, end_ind, form,lemma,tag,qualifiers,_))
  return data

In [77]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
list_morf = morfeusz_create_morf_table(df_2018, data_2018)
df_2018_morf = pd.DataFrame(list_morf, columns=cols)
df_2018_morf.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0,1,Zagospodarowanie,zagospodarować,ger:sg:nom.acc:n:perf:aff,[],[]
1,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",1,2,terenu,teren,subst:sg:gen:m3,[nazwa_pospolita],[]
2,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:acc,[],[]
3,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:dat,[],[]
4,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:loc,[],[]
5,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",3,4,bytym,bytym,ign,[],[]
6,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",4,5,baraku,barak,subst:sg:gen:m3,[nazwa_pospolita],[]
7,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",4,5,baraku,barak,subst:sg:loc:m3,[nazwa_pospolita],[]
8,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",4,5,baraku,barak,subst:sg:voc:m3,[nazwa_pospolita],[]
9,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",5,6,przy,przy,prep:loc,[],[]


In [78]:
df_2018_morf.shape

(122473, 10)

In [79]:
df_2018_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2018.csv', index=False)

## 2019

In [83]:
df_2019 = df[df['rok'] == 2019]
df_2019.head(10)

,rok,miasto,nazwa,czy_WAW
360,2019,GDYNIA,"Zakup nowoczesnego, W petni wyposazonego ambulansu ratunkowego Z zabudowq medycznq, zgodnego Z wymogami Ministerstwa Zdrowia, dla Miejskiej Stacji Pogotowia Ratunkowego W Gdyni",0.0
361,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0.0
362,2019,GDYNIA,"Ogrodzony wybieg dla psów, W poblizu schroniska dla zwierzgt Ciapkowo"" przy ulicach towicka - Olkuska W dzielnicy Maty Kack, dla psów Z catej Gdyni",0.0
363,2019,GDYNIA,Kampania edukacyjna dot. segregacji odpadów oraz mieciomaty/recyklomaty - urzadzenia do zdawania odpadów aluminiowych i szkta Z wymianq na bilety/znizki Z uzyciem Karty Mieszkanca,0.0
364,2019,GDYNIA,"Gdynskie taki kwietne zlokalizowane w 9 dzielnicach: Dqbrowa, Sródmiescie, Karwiny, Obtuze, Cisowa, Chylonia, Wielki Kack, Pogórze oraz Ortowo",0.0
365,2019,GDYNIA,Dostosowanie Estakady Kwiatkowskiego do ruchu rowerowego na odcinku Morska- Hutnicza,0.0
366,2019,GDYNIA,Mobilna stacja pomiarowa powietrza i obstugi interwencji zwiqzanych Z porzqdkiem publicznym (dron) oraz kampania informacyjna dot. zanieczyszczania powietrza,0.0
367,2019,GDYNIA,Organizacji pórkolonii letnich dla dzieci na terenie 23 szkót podstawowych W 21 dzielnicach Gdyni,0.0
368,2019,GDYNIA,"Ogrody deszczowe na petlach autobusowych/trolejbusowych (Chabrowa, Grabówek SKM, Obtuze Maciejewicza, Pogórze (ul. Czernickiego), Dqbrowa Mietowa, Karwiny Tesco, Ortowo SKM Klif)",0.0
369,2019,GDYNIA,Transport i wsparcie dla seniorów (65+) W dotarciu do miejsc uzytecznosci publicznej,0.0


In [84]:
data_2019 =[]

In [85]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
list_morf = morfeusz_create_morf_table(df_2019, data_2019)
df_2019_morf = pd.DataFrame(list_morf, columns=cols)
df_2019_morf.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,park,subst:pl:nom.acc.voc:m3,[nazwa_pospolita],[]
1,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:sg:gen:f,[nazwa_pospolita],[]
2,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:pl:nom.acc.voc:f,[nazwa_pospolita],[]
3,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:sg:gen:f,"[imię, nazwisko]",[]
4,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:pl:nom.acc.voc:f,"[imię, nazwisko]",[]
5,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sm1,subst:sg:gen:m1,[nazwisko],[]
6,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sm1,depr:pl:nom.acc.voc:m2,[nazwisko],[]
7,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",1,2,kieszonkowe,kieszonkowy,adj:pl:acc:m2.m3.f.n:pos,[],[]
8,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",1,2,kieszonkowe,kieszonkowy,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
9,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",1,2,kieszonkowe,kieszonkowy,adj:sg:acc:n:pos,[],[]


In [86]:
df_2019_morf.shape

(176322, 10)

In [87]:
df_2019_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2019.csv', index=False)

## 2020

In [88]:
df_2020 = df[df['rok'] == 2020]
df_2020.head(10)

,rok,miasto,nazwa,czy_WAW
0,2020,GDYNIA,Projektistatus,0.0
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0.0
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację,0.0
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",0.0
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną,0.0
5,2020,GDYNIA,Cotygodniowe weekendowe lekcje tańca towarzyskiego dla dorosłych,0.0
6,2020,GDYNIA,Meble miejskie zaprojektowane przez dzieci. Podziemia Hal Targowych w Gdyni.,0.0
7,2020,GDYNIA,"Odnowienie podwórek w obrębie ulic Płk. Dąbka, Kwiatkowskiego, Sucharskiego, Ledóchowskiego i Maciejewicza",0.0
8,2020,GDYNIA,Poprawa atrakcyjności skweru przy skrzyżowaniu Wielkopolska-Łowicka-Płocka przez nasadzenie roślin ozdobnych,0.0
9,2020,GDYNIA,Radarowe wyświetlacze prędkości na ul. Morskiej,0.0


In [89]:
data_2020 =[]

In [90]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
list_morf = morfeusz_create_morf_table(df_2020, data_2020)
df_2020_morf = pd.DataFrame(list_morf, columns=cols)
df_2020_morf.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,[nazwa_pospolita],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,[nazwa_pospolita],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,[nazwa_pospolita],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]
5,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:inst,[],[]
6,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",3,4,Estakadą,estakada,subst:sg:inst:f,[nazwa_pospolita],[]
7,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",4,5,Kwiatkowskiego,Kwiatkowski:Sm1,subst:sg:gen.acc:m1,[nazwisko],[]
8,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:C,conj,[],[]
9,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:I,interj,[],[]


In [91]:
df_2020_morf.shape

(183308, 10)

In [92]:
df_2020_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2020.csv', index=False)